In [8]:
# coding: utf8
import requests
import time
import re
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
from tqdm import tqdm
from datetime import date

In [9]:
def get_count(soup):
    """Возвращает кол-во объектов на сайте (т.е. то кол-во, которое они пишут)"""
    try:
        count = soup.find('div', class_='count')
        count = int(count.text.split(':')[1].strip())
        return count
    except:
        print('adds count non find')
        return None    

In [10]:
SEGMENT_LINK = {'Квартиры':'https://www.gipernn.ru/prodazha-kvartir?per-page=50'}

Получим начальную страницу, смотрим на кол-во объявлений, вычисляем кол-во страниц

In [11]:
url = SEGMENT_LINK['Квартиры']
r = requests.get(url)#получим html страницу
soup = BeautifulSoup(r.text, 'html.parser')#структурируем, у BeautifulSoup есть готовые методы для работы со структурой html 
count = get_count(soup)
print(f'всего {count} объявлений')
pagination_count = int(np.ceil(count/50))#кол-во страниц
add_links = []# список для ссылок

всего 6749 объявлений


Переходим по страницам и собираем ссылки. (если будут блакировки можно задать time.sleep(seconds))

In [12]:
for page in tqdm(range(1,pagination_count+1)):
#     time.sleep(0.5)
    url = f"{SEGMENT_LINK['Квартиры']}&page={page}"
    r = requests.get(url)
    soup = BeautifulSoup(r.text, 'html.parser')
    adds = soup.find('tbody').find_all('tr')
    for add in adds:
        link = add.find('a')
        if link:
            link = link.get('href')# ссылка на само объявление
            link = 'https://www.gipernn.ru'+link
            add_links.append(link)
    add_links = list(set(add_links))# only unique links
print('Кол-во ссылок', len(add_links))   

  0%|          | 0/135 [00:00<?, ?it/s]

Кол-во ссылок 5756


Парсим сами данные

In [ ]:
df = pd.DataFrame() #все данные собираем в датафрейм

In [ ]:
for link in tqdm(add_links):
#     time.sleep(0.5)
    r = requests.get(link)
    soup = BeautifulSoup(r.text, 'html.parser')
    
    d = {}#промежуточные данные храним в словаре 
    d['Ссылка'] = link    
    header = soup.find('h1')
    if header:
        header = header.text.strip().replace('\xa0', ' ')
    description = soup.find('div', class_='formatted-text')
    if description:
        description = description.text.strip().replace('\xa0', ' ')
    
    d['Заголовок объявления'] = header
    d['Описание'] = description
    try:    
        flat_feature_table = soup.find('table', id='w0')
        if flat_feature_table:
            flat_feature_table = flat_feature_table.find_all('tr')[1:]
            for row in flat_feature_table:
                key = row.find('th').text.strip()
                value = row.find('td').text.strip()
                d[key] = value
    except:
        print('table w0 no find')
    try:
        obj_feature_table = soup.find('table', id='w2')
        if obj_feature_table:
            obj_feature_table = obj_feature_table.find_all('tr')
            for row in obj_feature_table:
                key = row.find('th').text.strip()
                value = row.find('td').text.strip()
                if key not in d.keys():
                    d[key] = value
    except:
        print('table w2 no find')
    try:
        price = soup.find('div','offer-info').find('div', 'price').text.replace('\xa0', ' ').strip().split('  ')[0]
        d['Цена предложения, руб.'] = price
    except:
        print('price no find')
    try:
        properties = soup.find('div','properties m-t-2')    
        properties = properties.find_all('div',class_='property')
        for prop in properties:
            key = prop.find('div', 'property-name').text.strip()
            value = prop.find('div', 'property-value').text.strip()
            d[key] = value  
    except:
        print('properties m-t-2 no find')
    try:
        point_dists = soup.find('div', 'around').find_all('div', 'group')
        for point_dist in point_dists:
            name = point_dist.find('div','group-body').find('span').text
            dist = point_dist.find('div','group-body').find('div', 'company').find('span', 'company__distance').text
            d[name] = dist   
    except:
#         print('point-dist no find', link)
        pass
        
    df = df.append(d, ignore_index = True)  
    
df['Источник'] = 'gipernn'
df['Сегмент'] = 'Жилая недвижимость'
df['Подсегмент'] = 'Квартиры'
df['Тип рынка'] = 'Вторичный'# чтобы паристь первычный рынок, на сайте нужно ставить фильтр "в новостройке"
df['Город'] = 'Нижний Новгород'
df['Дата парсинга'] = str(date.today())

  0%|          | 0/5149 [00:00<?, ?it/s]

In [ ]:
df.head()

,Ссылка,Заголовок объявления,Описание,Район,Адрес,Комнат,Площадь,Этаж / этажность,Год постройки,Материал стен,...,Интернет,Телефон,Торг уместен,Тип комнат,Источник,Сегмент,Подсегмент,Тип рынка,Город,Дата парсинга
0,https://www.gipernn.ru/prodazha-kvartir/1-komn...,"Продажа 1-комнатной квартиры посёлок Дубёнки, ...",Продается 1-комнатная квартира в Советском рай...,Приокский,"посёлок Дубёнки, ул. Карбышева, д. 1",1 комната,"29,5 / 18 / 5,3",9 / 9,1975,кирпич,...,NaN,NaN,NaN,NaN,gipernn,Жилая недвижимость,Квартиры,Вторичный,Нижний Новгород,2022-09-29
1,https://www.gipernn.ru/prodazha-kvartir/3-komn...,Продажа 3-комнатной квартиры на ул. Маршала Ма...,Продаём замечательную 3х комнатную квартиру на...,Советский,"ул. Маршала Малиновского, д. 5",3 комнаты,61 / 44 / 7,2 / 9,1976,панель,...,NaN,NaN,NaN,NaN,gipernn,Жилая недвижимость,Квартиры,Вторичный,Нижний Новгород,2022-09-29
2,https://www.gipernn.ru/prodazha-kvartir/1-komn...,Продажа 1-комнатной квартиры на просп. Гагарин...,Продается однокомнатная квартира с очень хорош...,Приокский,"просп. Гагарина, д. 84",1 комната,"31 / 18 / 6,5",4 / 5,1961,кирпич,...,NaN,NaN,NaN,NaN,gipernn,Жилая недвижимость,Квартиры,Вторичный,Нижний Новгород,2022-09-29
3,https://www.gipernn.ru/prodazha-kvartir/3-komn...,Продажа 3-комнатной квартиры на ул. Верхнепечё...,Продаю Уютную 3-х комнатную квартиру в Нижегор...,Нижегородский,"ул. Верхнепечёрская, д. 4",3 комнаты,"65,1 / 38,6 / 9,7",6 / 9,1982,панель,...,NaN,NaN,NaN,NaN,gipernn,Жилая недвижимость,Квартиры,Вторичный,Нижний Новгород,2022-09-29
4,https://www.gipernn.ru/prodazha-kvartir/1-komn...,Продажа 1-комнатной квартиры на просп. Гагарин...,"Дом в ЖК ""Гагаринский высоты"". Во дворе д/са...",Приокский,"просп. Гагарина, д. 101 к5",1 комната,"36,3 / 21 / 6",9 / 10,2013,блок+утеплитель,...,NaN,NaN,NaN,NaN,gipernn,Жилая недвижимость,Квартиры,Вторичный,Нижний Новгород,2022-09-29


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 50 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   Ссылка                  1000 non-null   object
 1   Заголовок объявления    1000 non-null   object
 2   Описание                973 non-null    object
 3   Район                   1000 non-null   object
 4   Адрес                   1000 non-null   object
 5   Комнат                  1000 non-null   object
 6   Площадь                 1000 non-null   object
 7   Этаж / этажность        1000 non-null   object
 8   Год постройки           806 non-null    object
 9   Материал стен           1000 non-null   object
 10  Высота потолков         985 non-null    object
 11  Этажность               763 non-null    object
 12  Средняя цена за м2      760 non-null    object
 13  Подъездов               743 non-null    object
 14  Квартир                 764 non-null    object
 15  Школа

In [ ]:
df.shape

(1000, 50)

In [ ]:
df.to_excel('data/nn_flat.xlsx', index=False)
# df.to_csv('nn_flat.csv', index=False)
# df.to_sql()